In [1]:
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Initialize the Chrome webdriver
s = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

# Enter the URL
browser.get("https://www.charitiesnys.com/RegistrySearch/search_charities.jsp")

# Identify the EIN input element and send keys
ein_input = browser.find_element(
    By.XPATH, '//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]',
)
ein_input.send_keys("0")

# Click the search button
search_button = browser.find_element(
    By.XPATH, '//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]',
)
search_button.click()

# Allow time for the page to load
time.sleep(4)

# Identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR, "table.Bordered")

# Create an empty list to store data
data = []

# Loop through each row in the table
for i in range(1, 6):
    page = browser.find_element(
        By.XPATH, "/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[" + str(i) + "]",
    )
    for row in table.find_elements(By.CSS_SELECTOR, "tr"):
        cols = [cell.text for cell in row.find_elements(By.CSS_SELECTOR, "td")]
        data.append(cols)

# Update the column names
column_names = ["Organization Name",
                "NY Reg #",
                "EIN",
                "Registrant Type",
                "City",
                "State"]

# Create DataFrame
df = pd.DataFrame(data, columns=column_names)
# converting data frame to csv and saving it locally
df.to_csv("my_charities_data.csv", index=False)

/var/folders/wt/93lbgcg95kd4fd7p6x3w9xgr0000gn/T/ipykernel_65964/859661688.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import boto3
# Create a S3 client using the default credential provider chain
s3 = boto3.client("s3")

# Define bucket name
bucket_name = "my-m10-webscrap-charities-bucket"
region_name = "us-east-1"  # Specify the desired region

try:
    # Create the bucket without specifying the location constraint
    response = s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created successfully in region '{region_name}'.")
except Exception as e:
    print(f"Failed to create bucket: {e}")

Bucket 'my-m10-webscrap-charities-bucket' created successfully in region 'us-east-1'.


In [4]:
import time

# File to upload
file_path = "my_charities_data.csv"

# Generate timestamp
timestamp = time.strftime("%Y%m%d%H%M%S")

# Destination key with timestamp
destination_key = f"charities_bureau_scrape_{timestamp}.csv"  # Updated filename with timestamp

try:
    # Upload the file to the S3 bucket
    response = s3.upload_file(file_path, bucket_name, destination_key)
    print(
        f"File '{file_path}' uploaded successfully to '{bucket_name}' as '{destination_key}'."
    )
except Exception as e:
    print(f"Failed to upload file: {e}")

File 'my_charities_data.csv' uploaded successfully to 'my-m10-webscrap-charities-bucket' as 'charities_bureau_scrape_20240415130036.csv'.
